## Standard Bank Tech Impact Challenge: Xente credit scoring challenge

Remember to read the Zindi Competition details in the zindi platform, [Zindi Africa](https://zindi.africa/competitions/sbtic-xente-credit-scoring-challenge)

In [2]:
# import important modules 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns  
import warnings
warnings.filterwarnings('ignore')
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from boruta import BorutaPy  
from sklearn.feature_selection import SelectKBest 
from sklearn.feature_selection import chi2 
%matplotlib inline 

In [3]:
## reading the files and loading them into dataframes.
train = pd.read_csv('../data/Train.csv')
test= pd.read_csv('../data/Test.csv')
#sample = pd.read_csv('../../data/sample_submission.csv')
mask = pd.read_csv('../data/unlinked_masked_final.csv')
variabs = pd.read_csv('../data/VariableDefinitions.csv')

In [4]:
#check missing values 
print(train.isnull().sum())

CustomerId                 0
TransactionStartTime       0
Value                      0
Amount                     0
TransactionId              0
BatchId                    0
SubscriptionId             0
CurrencyCode               0
CountryCode                0
ProviderId                 0
ProductId                  0
ProductCategory            0
ChannelId                  0
TransactionStatus          0
IssuedDateLoan           612
AmountLoan               612
Currency                 612
LoanId                   612
PaidOnDate               612
IsFinalPayBack           612
InvestorId               612
DueDate                  614
LoanApplicationId        617
PayBackId                612
ThirdPartyId             614
IsThirdPartyConfirmed    612
IsDefaulted              612
dtype: int64


In [6]:
## EXPLORATORY DATA ANALYSIS 

#import the package 
import pandas_profiling


In [8]:
# generate report 
eda_report = pandas_profiling.ProfileReport(train)

In [9]:
# export into html fie 
eda_report.to_file("eda_report.html")

In [85]:
## FEATURE ENGINEERING 

# import category encoder 
import category_encoders as ce

## Transform dates types from 'object' to 'datetime'
train.TransactionStartTime=pd.to_datetime(train.TransactionStartTime)
test.TransactionStartTime=pd.to_datetime(test.TransactionStartTime)
train.IssuedDateLoan=pd.to_datetime(train.IssuedDateLoan)
test.IssuedDateLoan=pd.to_datetime(test.IssuedDateLoan)
train.PaidOnDate=pd.to_datetime(train.PaidOnDate)
train.DueDate=pd.to_datetime(train.DueDate)

## creating variables to transfer the information contained in the rows of the same transaction.
train['Number_Of_Split_Payments'] = 0 ## this is a count on the number of payments on the same loan. It will take a 0 for singled-rowed transactions, 1+ for multi-row transacs.
#train['Sum_Diff_Time_Payments'] = 0 ## I'm thinking of summing the delays between all payments made on a loan. It will take 0 for loans paid in a single time, 1+ for multiple payments on the same loan.
test['Number_Of_Split_Payments']=0
#test['Sum_Diff_Time_Payments']=0

## creating the feature : number of split payments on a loan.
train['Number_Of_Split_Payments']=train['TransactionId'].map(train.groupby('TransactionId').agg('count')['Number_Of_Split_Payments'])
test['Number_Of_Split_Payments']=test['TransactionId'].map(test.groupby('TransactionId').agg('count')['Number_Of_Split_Payments'])

train.drop(train[(train.TransactionId=='TransactionId_703')|((train.TransactionId=='TransactionId_927'))].index,axis=0,inplace=True)

## Lets drop the duplicate rows with the same transaction ID and keep the last one. (as in with the latest payment installment )
train.drop_duplicates(subset=['TransactionId'],keep='last',inplace=True)
test.drop_duplicates(subset=['TransactionId'],keep='last',inplace=True)

train.drop(['CountryCode','Currency','CurrencyCode','SubscriptionId','ProviderId','ChannelId'],axis=1,inplace=True)
test.drop(['CountryCode','CurrencyCode','SubscriptionId','ProviderId','ChannelId'],axis=1,inplace=True)

# Feature Engineering 
train['Count_Rejected_Loans'] = train['CustomerId'].map(train[train.TransactionStatus==0].groupby('CustomerId').LoanId.size())
test['Count_Rejected_Loans'] = test['CustomerId'].map(train[train.TransactionStatus==0].groupby('CustomerId').LoanId.size())
## then we should impute the columns of customers that were not found in the rejected list with 0 as in they have never been rejected.
train.Count_Rejected_Loans.fillna(value=0,inplace=True)
test.Count_Rejected_Loans.fillna(value=0,inplace=True)

## group train/test together to perform cumulative count
all_data=pd.concat((train,test))
## Initialize and compute values for the new feature
all_data['Cumulative_Reject']=0
all_data.loc[all_data.TransactionStatus==0,'Cumulative_Reject'] = all_data[all_data.TransactionStatus==0].groupby('CustomerId').cumcount()
## Separate all_data into train and test
train1=all_data[:len(train)]
test1=all_data[len(train):]
train['Cumulative_Reject']=0
test['Cumulative_Reject']=0
train['Cumulative_Reject']=train1['Cumulative_Reject']
test['Cumulative_Reject']=test1['Cumulative_Reject']

purchasestats=train[train.TransactionStatus==0].groupby('CustomerId').Value.agg(('mean','std','min','max'))
train['prchs_mean']=train['CustomerId'].map(purchasestats['mean'])
train['prchs_std']=train['CustomerId'].map(purchasestats['std'])
train['prchs_max']=train['CustomerId'].map(purchasestats['max'])
train['prchs_min']=train['CustomerId'].map(purchasestats['min'])
test['prchs_mean']=test['CustomerId'].map(purchasestats['mean'])
test['prchs_std']=test['CustomerId'].map(purchasestats['std'])
test['prchs_max']=test['CustomerId'].map(purchasestats['max'])
test['prchs_min']=test['CustomerId'].map(purchasestats['min'])

valuegroups=mask.groupby('CustomerId').Value.agg(('mean','std','min','max'))
train['mean_cus_transac']=train['CustomerId'].map(valuegroups['mean'])
train['std_cus_transac']=train['CustomerId'].map(valuegroups['std'])
train['min_cus_transac']=train['CustomerId'].map(valuegroups['min'])
train['max_cus_transac']=train['CustomerId'].map(valuegroups['max'])
test['mean_cus_transac']=test['CustomerId'].map(valuegroups['mean'])
test['std_cus_transac']=test['CustomerId'].map(valuegroups['std'])
test['min_cus_transac']=test['CustomerId'].map(valuegroups['min'])
test['max_cus_transac']=test['CustomerId'].map(valuegroups['max'])

train['Day_Of_Week']= train.TransactionStartTime.dt.weekday
test['Day_Of_Week'] =test.TransactionStartTime.dt.weekday
train['Day_in_month']=train.TransactionStartTime.dt.day
test['Day_in_month']=test.TransactionStartTime.dt.day

from datetime import date
datemin = date(2018,9,21)
datemax= date(2019,7,17)
(datemax-datemin).days
datesinc=pd.DataFrame(columns=['date','inc_value'])
datesinc.loc[0,'inc_value']=1
datesinc.loc[0,'date']=datemin
from datetime import timedelta
for i in range(2,301):
    datesinc.loc[i-1,'inc_value']=i
    datesinc.loc[i-1,'date']=datemin + timedelta(days=i-1)
train['inc_value_date']=train.TransactionStartTime.dt.date.map(datesinc.set_index('date').inc_value)
test['inc_value_date']=test.TransactionStartTime.dt.date.map(datesinc.set_index('date').inc_value)

train.inc_value_date = train.inc_value_date.astype(np.int64)
test.inc_value_date = test.inc_value_date.astype(np.int64)

aa=train[(train.TransactionStatus==1)&(train.TransactionStartTime<train.DueDate)].groupby('CustomerId').agg(('count','mean','std','min','max')).Value
#train['number_transac_before_due']=train['CustomerId'].map(aa['count'])
train['before_due_mean'] = train['CustomerId'].map(aa['mean'])
train['before_due_std'] = train['CustomerId'].map(aa['std'])
train['before_due_min'] = train['CustomerId'].map(aa['min'])
train['before_due_max'] = train['CustomerId'].map(aa['max'])
test['before_due_mean'] = test['CustomerId'].map(aa['mean'])
test['before_due_std'] = test['CustomerId'].map(aa['std'])
test['before_due_min'] = test['CustomerId'].map(aa['min'])
test['before_due_max'] = test['CustomerId'].map(aa['max'])


train['Cnt_missed_payment']=0
train.loc[train.DueDate<train.PaidOnDate,'Cnt_missed_payment']=train[train.DueDate<train.PaidOnDate].groupby('CustomerId').cumcount()
test['Cnt_missed_payment']=test['CustomerId'].map(train.groupby('CustomerId').agg('max').Cnt_missed_payment)

train=train[train.IsDefaulted.notnull()]

features = ['CustomerId', #'TransactionStartTime', 
            'Value', #'Amount',
       #'TransactionId', #'BatchId', 
             'ProductId',
       'ProductCategory', #'TransactionStatus', 
            #'IssuedDateLoan',
       #'LoanId', 'InvestorId', 'LoanApplicationId', 'ThirdPartyId',
       'Number_Of_Split_Payments', 'Count_Rejected_Loans', 
           #'Cumulative_Reject',
       #'prchs_mean', 'prchs_std', 'prchs_max', 'prchs_min', 
            'mean_cus_transac','std_cus_transac', 'min_cus_transac', 'max_cus_transac', 
            'Day_Of_Week','Day_in_month', 'inc_value_date', 'before_due_mean', 'before_due_std',
       'before_due_min', 'before_due_max', 
            'Cnt_missed_payment'
]


oce = ce.OneHotEncoder(cols=['ProductId','ProductCategory'])
tce = ce.TargetEncoder(cols=['CustomerId'],smoothing=40,min_samples_leaf=3)

X = train[features]

#handle missing values in training set
X = X.fillna(X.median())

X_test = test[features]
#handle missing values in training set
X_test = X_test.fillna(X_test.median())

y=train.IsDefaulted.copy()
X = oce.fit_transform(X)
X = tce.fit_transform(X,y)
X_test = oce.transform(X_test)
X_test = tce.transform(X_test)


#FEATURE SELECTION

#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=25)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(25,'Score'))  #print 10 best features

best_features_df = featureScores.nlargest(25,'Score')

best_features_cols = list(best_features_df.Specs)

                       Specs         Score
35            before_due_min  1.036756e+08
1                      Value  7.545044e+07
33           before_due_mean  7.270749e+07
36            before_due_max  4.363295e+07
29           max_cus_transac  1.241756e+06
27           std_cus_transac  2.044060e+05
28           min_cus_transac  5.541087e+04
34            before_due_std  3.371891e+04
26          mean_cus_transac  1.988354e+04
15              ProductId_14  5.532635e+02
22         ProductCategory_6  5.532635e+02
32            inc_value_date  2.737818e+02
24  Number_Of_Split_Payments  8.675770e+01
25      Count_Rejected_Loans  8.099436e+01
37        Cnt_missed_payment  2.716352e+01
17         ProductCategory_1  2.102796e+01
30               Day_Of_Week  1.003593e+01
5                ProductId_4  9.948098e+00
2                ProductId_1  9.885831e+00
18         ProductCategory_2  9.190848e+00
0                 CustomerId  7.415284e+00
3                ProductId_2  4.348468e+00
6          

In [86]:
#select best features 

X = X[best_features_cols]
X_test = X_test[best_features_cols]

In [87]:
#split into train and valid 
X_Train, X_val, y_Train, y_val = train_test_split(X,y, test_size=0.05,random_state=42)

In [100]:
## NEW CLASIFIERS FROM SCIKIT LEARN
# explicitly require this experimental feature
from sklearn.experimental import enable_hist_gradient_boosting 

# now you can import normally from ensemble 
from sklearn.ensemble import HistGradientBoostingClassifier 


In [90]:
pipeline_optimizer = HistGradientBoostingClassifier() 

parameters = {
    'max_iter': [100,200,500,700,1000,1200,1500],
    'learning_rate': [0.1,0.05,1],
    'max_depth' : [25, 50, 75],
    'l2_regularization': [1.5],
    'scoring': ['f1_micro','balanced_accuracy','roc_auc'],
    'random_state' : [42],
    'verbose':[2],
    }



pipeline_optimizer_grid = GridSearchCV(pipeline_optimizer, parameters, n_jobs=2, 
                   cv=5, scoring='roc_auc',
                   verbose=2, refit=True)

In [91]:
# handle imbalance of data in the trainset 
from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=42)
X_train, y_train = sm.fit_resample(X_Train, y_Train)

# convert into Dataframe
features = list(X.columns)

X_train = pd.DataFrame(X_train, columns=features)

In [101]:
#training 
pipeline_optimizer_grid.fit(X_train, y_train)


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               learning_rate=0.1, max_depth=-1, min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_leaves=31, objective=None, random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=1)

In [102]:
#find the performance
print(pipeline_optimizer_grid.score(X_val, y_val))


0.9482758620689655


In [103]:
# Get the predicted result for the test Data
preds = pipeline_optimizer_grid.predict(X_test)

In [104]:
sample_submission = pd.DataFrame(columns=['TransactionId','IsDefaulted'])
sample_submission['TransactionId'] = test['TransactionId']
sample_submission['IsDefaulted'] = preds



In [105]:
# Create submission csv file
from datetime import datetime
now = datetime. now()
current_time = now.strftime("%S")

sample_submission.to_csv('../data/submissions/my_submission_{}_sun.csv'.format(current_time), index = False)